## Authors:
- Jakub Janicki
- Kuba Kondracki
- Hubert Twardowski

In [1]:
import transformers
import evaluate
import numpy as np
import os
import pandas as pd
import pickle
from nlp import Dataset

c:\Users\janic\anaconda3\envs\dnn\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Data setup

In [2]:
ALBERT_MODEL = 'albert-base-v2'    # https://huggingface.co/albert/albert-base-v2
tokenizer = transformers.AlbertTokenizer.from_pretrained(ALBERT_MODEL)
    
def encode_batch(batch):
    return tokenizer(batch['text_combined'], padding=True, truncation=True, max_length=512)

# precompute the train and test datasets
if os.path.exists('train_dataset.pkl') and os.path.exists('test_dataset.pkl'):
    with open('train_dataset.pkl', 'rb') as f:
        train_dataset = pickle.load(f)
    with open('test_dataset.pkl', 'rb') as f:
        test_dataset = pickle.load(f)
else:
    df = pd.read_csv('phishing_email.csv')[['text_combined', 'label']].dropna()
    dataset = Dataset.from_pandas(df)
    data_dct = dataset.train_test_split(train_size=0.7)
    train_dataset = data_dct['train'].map(encode_batch, batched=True)
    test_dataset = data_dct['test'].map(encode_batch, batched=True)
    with open('train_dataset.pkl', 'wb') as f:
        pickle.dump(train_dataset, f)
    with open('test_dataset.pkl', 'wb') as f:
        pickle.dump(test_dataset, f)

train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

In [3]:
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)


## Model setup

In [ ]:
id2label = {0: "SAFE", 1: "PHISHING"}
label2id = {"PHISHING": 1, "SAFE": 0}
model = transformers.AutoModelForSequenceClassification.from_pretrained(
    f"albert/{ALBERT_MODEL}", num_labels=2, id2label=id2label, label2id=label2id, attn_implementation="sdpa")
# SDPA also from ALBERT usage guide

training_args = transformers.TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    auto_find_batch_size=True,
    dataloader_num_workers=4,
     # this model is too large to tune hyperparameters. 
     # lr of 1e-5 has always worked fine with models we've trained previously, with some wd like 1e-8 or 1e-10 preventing explosions.
     # instead of hparam tuning, we decided to use lr scheduler to diminish the impact of incorrect lr.
    learning_rate=1e-5,
    lr_scheduler_type="cosine_with_min_lr",
    lr_scheduler_kwargs={"min_lr": 1e-8},
    weight_decay=1e-8,
    num_train_epochs=2,
    logging_dir="./logs",
    load_best_model_at_end=True,
    logging_steps=100,
    eval_strategy="steps",
    save_strategy="steps",
    save_steps=500, # checkpoint every 500 steps
    bf16=True, # bfloat16 since im getting {"grad_norm": nan} with float16 and i cant quite run float32
    report_to="tensorboard", # setup tensorboard
    max_steps=1000, # limit the number of steps to 1000
)

trainer = transformers.Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert/albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
max_steps is given, it will override any value given in num_train_epochs


## Sanity check

In [11]:
print(f"Training dataset size: {len(train_dataset)}")
print(f"Test dataset size: {len(test_dataset)}")
print(f"Training label distribution: {sum(train_dataset['label']) / len(train_dataset['label'])}")
print(f"Test label distribution: {sum(test_dataset['label']) / len(test_dataset['label'])}")

print(f"First train sample: {train_dataset[0]}")
print(f"Model architecture: {model}")

Training dataset size: 65988
Test dataset size: 16498
Training label distribution: 0.521534218342729
Test label distribution: 0.5137592435446721
First train sample: {'label': tensor(0), 'input_ids': tensor([    2,  2247,  1266,   289,   530,  4696,   315,   169,   883, 21156,
          228,   286,   883,  7681, 21156,   315,   169,  1067,   296,   303,
         1100,   148, 11597,  7912,   315,   169,  2094,   883,  4247,   169,
          883,   289,   913,  2824,   193,  1015,  3062,   138,  1100,   148,
        11597,  7912,  1364,   265,   883,  1466,   868,    83,  1466,   651,
         1100,   148,   289,  3553,  1466, 12238, 21156,  2945,  7681, 21156,
         1100,   148,  7912,    43,  1148,  9021,   442,  6305,  1696, 21156,
          883,   169,    87,   910,   193,   868,  2670, 21156,  3081,   955,
          529,  4466,  2383,  1139,   341, 14403,  5457,   609,  1131,  2247,
         1266,   530,   134,  1832,    28,    18,   254,    34,    62,     3,
            0,     0,

In [6]:
print(f"TensorBoard logs will be stored in: {training_args.logging_dir}")

TensorBoard logs will be stored in: ./logs


In [7]:
baseline_metrics = trainer.evaluate()
print(f"Baseline metrics: {baseline_metrics}")

100%|██████████| 2063/2063 [02:47<00:00, 12.32it/s]

Baseline metrics: {'eval_loss': 0.715407133102417, 'eval_model_preparation_time': 0.0, 'eval_accuracy': 0.51709298096739, 'eval_runtime': 174.6065, 'eval_samples_per_second': 94.487, 'eval_steps_per_second': 11.815}


In [12]:
# check for correct padding

# https://huggingface.co/docs/transformers/model_doc/albert usage tip:
# ALBERT is a model with absolute position embeddings so it’s usually advised to pad the inputs on the right rather than the left.
train_dataset[0]

{'label': tensor(0),
 'input_ids': tensor([    2,  2247,  1266,   289,   530,  4696,   315,   169,   883, 21156,
           228,   286,   883,  7681, 21156,   315,   169,  1067,   296,   303,
          1100,   148, 11597,  7912,   315,   169,  2094,   883,  4247,   169,
           883,   289,   913,  2824,   193,  1015,  3062,   138,  1100,   148,
         11597,  7912,  1364,   265,   883,  1466,   868,    83,  1466,   651,
          1100,   148,   289,  3553,  1466, 12238, 21156,  2945,  7681, 21156,
          1100,   148,  7912,    43,  1148,  9021,   442,  6305,  1696, 21156,
           883,   169,    87,   910,   193,   868,  2670, 21156,  3081,   955,
           529,  4466,  2383,  1139,   341, 14403,  5457,   609,  1131,  2247,
          1266,   530,   134,  1832,    28,    18,   254,    34,    62,     3,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,   

## Training

In [6]:
# train
trainer.train()
trainer.evaluate()
trainer.save_model("albert-base-v2-phishing-emails")

 10%|█         | 100/1000 [00:32<03:38,  4.12it/s]

{'loss': 0.4846, 'grad_norm': 64.0195083618164, 'learning_rate': 9.755527298894292e-06, 'epoch': 0.01}


                                                  
 10%|█         | 100/1000 [03:28<03:38,  4.12it/s] 

{'eval_loss': 0.2660793662071228, 'eval_accuracy': 0.9004727845799491, 'eval_runtime': 176.0971, 'eval_samples_per_second': 93.687, 'eval_steps_per_second': 11.715, 'epoch': 0.01}


 20%|██        | 200/1000 [03:52<03:15,  4.10it/s]   

...

100%|██████████| 1000/1000 [30:29<00:00,  4.09it/s] 

{'loss': 0.1602, 'grad_norm': 0.3658551275730133, 'learning_rate': 1e-08, 'epoch': 0.12}


                                                   
100%|██████████| 1000/1000 [33:24<00:00,  2.00s/it]


{'eval_loss': 0.16470371186733246, 'eval_accuracy': 0.9627227542732453, 'eval_runtime': 174.9576, 'eval_samples_per_second': 94.297, 'eval_steps_per_second': 11.791, 'epoch': 0.12}
{'train_runtime': 2004.5988, 'train_samples_per_second': 3.991, 'train_steps_per_second': 0.499, 'train_loss': 0.27497378349304197, 'epoch': 0.12}


100%|██████████| 2063/2063 [02:47<00:00, 12.31it/s]


## Results

In [ ]:
results = trainer.evaluate()
print(results)

100%|██████████| 2063/2063 [02:48<00:00, 12.27it/s]

{'eval_loss': 0.16470371186733246, 'eval_accuracy': 0.9627227542732453, 'eval_runtime': 175.2363, 'eval_samples_per_second': 94.147, 'eval_steps_per_second': 11.773, 'epoch': 0.12122681537156019}
